In [13]:
import cv2
import numpy
import typing
import logging
import gevent.monkey
import gevent
import sys
import traceback
from skimage.feature import haar_like_feature
args = sys.argv

from utils import create_folder, get_logger
logger: logging.Logger = get_logger(__name__)
gevent.monkey.patch_all()
"""
    Part 1
"""
def convolution_2D(src: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:
    image_height, image_width = src.shape
    kernel_height, kernel_width = kernel.shape
    output = numpy.zeros(( image_height - kernel_height + 1, image_width - kernel_width + 1))
    for i in range(image_height - kernel_height + 1):
        for j in range(image_width - kernel_width + 1):
            output[i, j] = numpy.sum(numpy.multiply(src[i:i+kernel_height, j:j+kernel_width], kernel))
    return output

def gaussian_2d_convolution(size:int, sigma: float=1.0)-> numpy.ndarray:
    gussian_1d = cv2.getGaussianKernel(ksize=size, sigma=sigma)
    # logger.info(f"size:{size}, gussian_1d:{gussian_1d}")
    return numpy.outer(a=gussian_1d, b=gussian_1d)



convolut_kernel_dict = {
    "sobel_kernel_x_3": numpy.array(object=[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=float),
    "sobel_kernel_y_3": numpy.array(object=[[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=float),
    "sobel_kernel_x_5": numpy.array(object=[[-1,-2, 0, 2, 1], [-2, -3, 0 ,3 ,2], [0, 0, 0 , 0, 0], [2,3,0,-3,-2], [1,2,0,-2,-1]], dtype=float),
    "sobel_kernel_y_5": numpy.array(object=[[-1,-4, -6, -4, -1], [-2, -8, -12 ,-8 , -2], [0, 0, 0 , 0, 0], [2,8,12,8,2], [1,4,6,4,1]], dtype=float),
    "gaussian": gaussian_2d_convolution,
    "5-haar-like-masks": [numpy.array(object=[[1, -1], [1, -1]], dtype=float), 
                          numpy.array(object=[[1, 1], [-1, -1]], dtype=float), 
                          numpy.array(object=[[1, -1, 1], [1, -1, 1]], dtype=float), 
                          numpy.array(object=[[1, 1], [-1, -1], [1, 1]], dtype=float), 
                          numpy.array(object=[[1, -1], [-1, 1]], dtype=float)],
}
IMAGE_PATH = "einstein.png"
SIFT_IMAGE_PATH = "im01.jpg"
SOBEL_FOLDER_NAME = "sobel"
SOBEL_FOLDER_NAME2 = "sobel2"
GUSSIAN_FOLDER_NAME = "gaussian"
GUSSIAN_FOLDER_NAME2 = "gaussian2"
HAAR_MASK_NAME = "haar_mask"
HAAR_MASK_NAME2 = "haar_mask2"
# cv.IMREAD_COLOR cv.IMREAD_GRAYSCALE
origin_image: cv2.typing.MatLike = cv2.imread(filename=IMAGE_PATH, flags=cv2.IMREAD_GRAYSCALE)
logger.info(f"type:{type(origin_image)}")
logger.info(f"origin_image:{origin_image}")

# sobel kernel
sobel_size_arr: typing.List[int] = [3, 5]
create_folder(folder_name=SOBEL_FOLDER_NAME)
create_folder(folder_name=SOBEL_FOLDER_NAME2)
for sobel_size in sobel_size_arr:
    # use my custom 2d convolution

    sobel_x_result: numpy.ndarray = convolution_2D(src=origin_image, 
                                                  kernel=convolut_kernel_dict.get(f"sobel_kernel_x_{sobel_size}"))
    sobel_y_result: numpy.ndarray = convolution_2D(src=origin_image, 
                                                  kernel=convolut_kernel_dict.get(f"sobel_kernel_y_{sobel_size}"))
    sobel_combo_result:cv2.typing.MatLike = cv2.addWeighted(src1=sobel_x_result, alpha=0.5, src2=sobel_y_result, beta=1-0.5, gamma=1.0)
    create_folder(folder_name=f"{SOBEL_FOLDER_NAME}/size{sobel_size}")
    cv2.imwrite(f"{SOBEL_FOLDER_NAME}/size{sobel_size}/sobel_x_result.png", sobel_x_result)
    cv2.imwrite(f"{SOBEL_FOLDER_NAME}/size{sobel_size}/sobel_y_result.png", sobel_y_result)
    cv2.imwrite(f"{SOBEL_FOLDER_NAME}/size{sobel_size}/sobel_combo_result.png", sobel_combo_result)

    # use cv2.Sobel api
    sobel_x_result:cv2.typing.MatLike = cv2.Sobel(src=origin_image, ddepth=-1, dx=1, dy=0, ksize=sobel_size) 
    sobel_y_result:cv2.typing.MatLike = cv2.Sobel(src=origin_image, ddepth=-1, dx=0, dy=1, ksize=sobel_size) 
    sobel_combo_result:cv2.typing.MatLike = cv2.Sobel(src=origin_image, ddepth=-1, dx=1, dy=1, ksize=sobel_size)
    # the brightness is too low
    create_folder(folder_name=f"{SOBEL_FOLDER_NAME2}/size{sobel_size}")
    cv2.imwrite(f"{SOBEL_FOLDER_NAME2}/size{sobel_size}/sobel_x_result.png", sobel_x_result)
    cv2.imwrite(f"{SOBEL_FOLDER_NAME2}/size{sobel_size}/sobel_y_result.png", sobel_y_result)
    cv2.imwrite(f"{SOBEL_FOLDER_NAME2}/size{sobel_size}/sobel_combo_result.png", sobel_combo_result)

# Gaussian kernel

image_height, image_width = origin_image.shape
logger.info(f"shape:{origin_image.shape}")
max_gaussian_size:int = image_width // 16
create_folder(folder_name=GUSSIAN_FOLDER_NAME)
create_folder(folder_name=GUSSIAN_FOLDER_NAME2)


def guassian_convolution(image: cv2.typing.MatLike, guassian_size: int, group:int = 5 ):
    create_folder(folder_name=f"{GUSSIAN_FOLDER_NAME}/size{guassian_size}")
    create_folder(folder_name=f"{GUSSIAN_FOLDER_NAME2}/size{guassian_size}")

    scale_factor = 1
    # it's better to use √(1.6^2 - 0.5^2)
    sigma = 1.52
    resized_image = image
    # use my custom 2d convolution
    for it in range(1,group):
        new_width = int(image.shape[1] * scale_factor)
        new_height = int(image.shape[0] * scale_factor)
        scale_factor /= 2
        resized_image = cv2.resize(resized_image, (new_width, new_height))
        guassian_result: numpy.ndarray = convolution_2D(src=resized_image, 
                                         kernel=gaussian_2d_convolution(size=guassian_size, sigma=sigma))
        sigma *=2 
        cv2.imwrite(f"{GUSSIAN_FOLDER_NAME}/size{guassian_size}/guassian_result_it{it}.png", guassian_result)
    
    resized_image = image
    scale_factor = 1
    sigma = 1.52
    for it in range(1,group):
        new_width = int(image.shape[1] * scale_factor)
        new_height = int(image.shape[0] * scale_factor)
        scale_factor /= 2
        resized_image = cv2.resize(resized_image, (new_width, new_height))
        guassian_result:cv2.typing.MatLike = cv2.GaussianBlur(src=resized_image, 
                                                              ksize=(guassian_size, guassian_size), 
                                                              sigmaX=sigma, sigmaY=sigma, 
                                                              borderType=cv2.BORDER_ISOLATED)
        sigma *=2 
        cv2.imwrite(f"{GUSSIAN_FOLDER_NAME2}/size{guassian_size}/guassian_result_it{it}.png", guassian_result)

# log2(min(M,N)) -3
o = int(numpy.log2(min(image_height, image_width))) - 3
for guassian_size in range(3, max_gaussian_size, 2):
    coroutine = gevent.spawn(guassian_convolution, **{
        "image": origin_image,
        "guassian_size": guassian_size,
        "group": o
    })
def generate_5masks_by_scale(scale: int) -> typing.List[numpy.ndarray[float]]:
    if scale & 1 == 1:
        scale += 1
    if scale <= 2:
        return convolut_kernel_dict.get("5-haar-like-masks")
    edge_vertical = numpy.ones(shape=(scale, scale), dtype=float)
    edge_vertical[:, scale//2:] = -1.0
    edge_horizon = numpy.ones(shape=(scale, scale), dtype=float)
    edge_horizon[scale//2 : , :] = -1.0
    
    line_vertical = numpy.ones(shape=(scale, int(scale * 1.5)), dtype=float)
    line_vertical[:, scale //3 : min(2 * (scale //3) + 1, int(scale * 1.5) )]
    line_horizon = numpy.ones(shape=(int(scale * 1.5), scale), dtype=float)
    line_horizon[scale //3 : min(2 * (scale //3) + 1, int(scale * 1.5)), :]

    rectangle = numpy.ones(shape=(scale , scale), dtype=float)
    rectangle[:scale//2, scale//2:] = -1
    rectangle[scale//2:, : scale//2] = -1
    scaled_masks: typing.List[numpy.ndarray[float]] = [edge_vertical, edge_horizon, line_vertical, line_horizon, rectangle]
    return scaled_masks

create_folder(folder_name=HAAR_MASK_NAME)
for sacle in range(4, 36, 2):
    create_folder(folder_name=f"{HAAR_MASK_NAME}/sacle{sacle}")
    masks: typing.List[numpy.ndarray[float]] = generate_5masks_by_scale(sacle)
    create_folder(folder_name=f"{HAAR_MASK_NAME}/sacle{sacle}/edge_vertical")
    mask_result: numpy.ndarray = convolution_2D(src=origin_image, 
                                                  kernel=masks[0])
    cv2.imwrite(f"{HAAR_MASK_NAME}/sacle{sacle}/edge_vertical/mask_edge_vertical_result.png", mask_result)

    mask_result: numpy.ndarray = convolution_2D(src=origin_image, 
                                                kernel=masks[1])
    cv2.imwrite(f"{HAAR_MASK_NAME}/sacle{sacle}/edge_vertical/mask_edge_horizon_result.png", mask_result)
    mask_result: numpy.ndarray = convolution_2D(src=origin_image, 
                                                kernel=masks[2])
    cv2.imwrite(f"{HAAR_MASK_NAME}/sacle{sacle}/edge_vertical/mask_line_vertical_result.png", mask_result)
    mask_result: numpy.ndarray = convolution_2D(src=origin_image, 
                                                kernel=masks[3])
    cv2.imwrite(f"{HAAR_MASK_NAME}/sacle{sacle}/edge_vertical/mask_line_horizon_result.png", mask_result)
    mask_result: numpy.ndarray = convolution_2D(src=origin_image, 
                                                kernel=masks[4])
    cv2.imwrite(f"{HAAR_MASK_NAME}/sacle{sacle}/edge_vertical/rectangle_result.png", mask_result)

    """
    Conclusion:
        sobel kernel is used to detect the edges of Objects in the image by gradients
        gussian kernal is used to blur and smooth the images
        Haar-like Masks is used to detect the features of variations in blocks of an image

        The size of sobel kernel influences the size of edges which it can detect. When kernel is becoming larger, the smaller edges will not display in the output.
        The size of gussian kernel influences how smoothy the output is and how many info will lose in the original images. 
        The scale of  Haar-like Masks influences
    """

    image = cv2.imread(SIFT_IMAGE_PATH)
    sift = cv2.SIFT.create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    output_image = cv2.drawKeypoints(image=image, keypoints=keypoints, outImage=None, flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
    cv2.imwrite(f"SIFT.png", output_image)




    

    












2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>


2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,255 - __main__ - INFO - type:<class 'numpy.ndarray'>
INFO:__main__:type:<class 'numpy.ndarray'>
2023-09-27 02:36:46,692 - __main__ - INFO - origin_image:[[ 24  23  23 ...  28  2

ModuleNotFoundError: No module named '_tkinter'